# Consulta a la API de EMT de Madrid para obtener datos de parkings

La documentación de esta API está en esta dirección: https://apidocs.emtmadrid.es/

El primer paso antes de usar la API es registrarse en https://mobilitylabs.emtmadrid.es/ ya que para usar la API se necesita un token que se genera con el e-mail y la contraseña que hayamos puesto en el registro

In [0]:
import requests
import json



#Autenticación: 

url = "https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/"

headers = {
    'email': "mariceli.paredes@gmail.com",
    'password': "S86338633c"
    }


response = requests.get (url,headers=headers)

valid_token=json.loads(response.text)["data"][0]["accessToken"]


url = "https://openapi.emtmadrid.es/v1/citymad/places/parkings/es-ES/"

headers = {
    'accesstoken': valid_token

    }

response = requests.get(url, headers=headers)


data = response.json()



cantidad = len (data['data'][0]['data'])


ind=0
filep = open('parking.csv', 'w')

#Me voy a quedar con los siguientes datos:

#Nombre del parking
#AreaCode que es el código postal
#Coordenadas 
#Dirección

print("{};{};{};{};{}".format("id","nombre", "areacode", "coordenadas",
                             "direccion"), 
                             file=filep)

while (ind<cantidad):
  
  print("{};{};{};{};{}".format(ind+1, data['data'][0]['data'][ind]['name'], data['data'][0]['data'][ind]['areacode'], 
                             data['data'][0]['data'][ind]['geometry']['coordinates'],data['data'][0]['data'][ind]['address']), 
                             file=filep)
  ind=ind+1
  
filep.close()



data

#Habrá que realizar posteriormente una limpieza de datos ya que en muchos registros no hay código postal y podemos también 
#quitar los #símbolos "[" y "]" de las coordenadas geográficas


#data['data'][0]['data'][0]['name']
#data['data'][0]['data'][0]['areacode']
#data['data'][0]['data'][0]['geometry']['coordinates']
#data['data'][0]['data'][0]['address']



In [0]:
from google.colab import files
files.download('parking.csv')  

**Con el siguiente código podemos cargar directamente el fichero en el Google Storage**

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

In [10]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('parking.csv', 
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket='dataproc-fe7b85fd-43dc-4de0-a520-a824fcd432de-europe-west1', 
                                       name='parking.csv',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')

Upload complete
